In [9]:
import sys
sys.path.append('/fsx/matzeni/duck')

import torch
from einops import rearrange, repeat
from duck.box_tensors import BoxTensor
from matplotlib import pyplot as plt

In [10]:
a = torch.rand((4, 8))
b = torch.rand((6, 8))

In [11]:
max_rels = torch.tensor([a.size(0), b.size(0)]).max()

In [12]:
pad = torch.zeros((max_rels - a.size(0), a.size(1)), device=a.device)
mask_a = torch.cat([torch.ones_like(a), pad]).bool()
a = torch.cat([a, pad])

In [13]:
a

tensor([[0.5869, 0.3689, 0.9671, 0.6872, 0.4481, 0.8763, 0.2241, 0.7484],
        [0.9406, 0.8606, 0.8131, 0.7456, 0.8908, 0.8226, 0.6802, 0.3038],
        [0.4385, 0.6148, 0.5822, 0.6618, 0.5690, 0.2471, 0.6140, 0.0323],
        [0.4150, 0.9016, 0.1953, 0.1318, 0.6647, 0.4806, 0.9696, 0.8082],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])

In [14]:
mask_a

tensor([[ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False, False, False],
        [False, False, False, False, False, False, False, False]])

In [15]:
pad = torch.zeros((max_rels - b.size(0), b.size(1)), device=b.device)
mask_b = torch.cat([torch.ones_like(b), pad]).bool()
b = torch.cat([b, pad])

In [16]:
mask_b

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [17]:
b

tensor([[0.8015, 0.4549, 0.6941, 0.1677, 0.7015, 0.1137, 0.7601, 0.2935],
        [0.1857, 0.7594, 0.2515, 0.3138, 0.1438, 0.4808, 0.0077, 0.3284],
        [0.8238, 0.6393, 0.8214, 0.7430, 0.4867, 0.7158, 0.7789, 0.5869],
        [0.4623, 0.7353, 0.0265, 0.2426, 0.7059, 0.7859, 0.0518, 0.1387],
        [0.1538, 0.0783, 0.2859, 0.9093, 0.2125, 0.1035, 0.8887, 0.0912],
        [0.9264, 0.3494, 0.7079, 0.2979, 0.9180, 0.4977, 0.0101, 0.7014]])

In [18]:
ab = torch.stack([a, b])

In [19]:
ab

tensor([[[0.5869, 0.3689, 0.9671, 0.6872, 0.4481, 0.8763, 0.2241, 0.7484],
         [0.9406, 0.8606, 0.8131, 0.7456, 0.8908, 0.8226, 0.6802, 0.3038],
         [0.4385, 0.6148, 0.5822, 0.6618, 0.5690, 0.2471, 0.6140, 0.0323],
         [0.4150, 0.9016, 0.1953, 0.1318, 0.6647, 0.4806, 0.9696, 0.8082],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.8015, 0.4549, 0.6941, 0.1677, 0.7015, 0.1137, 0.7601, 0.2935],
         [0.1857, 0.7594, 0.2515, 0.3138, 0.1438, 0.4808, 0.0077, 0.3284],
         [0.8238, 0.6393, 0.8214, 0.7430, 0.4867, 0.7158, 0.7789, 0.5869],
         [0.4623, 0.7353, 0.0265, 0.2426, 0.7059, 0.7859, 0.0518, 0.1387],
         [0.1538, 0.0783, 0.2859, 0.9093, 0.2125, 0.1035, 0.8887, 0.0912],
         [0.9264, 0.3494, 0.7079, 0.2979, 0.9180, 0.4977, 0.0101, 0.7014]]])

In [20]:
mask = torch.stack([mask_a, mask_b])

In [21]:
mask

tensor([[[ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [False, False, False, False, False, False, False, False],
         [False, False, False, False, False, False, False, False]],

        [[ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True,  True]]])

In [23]:
ab = rearrange(ab, "w b d -> (w b) d")

In [25]:
mask = rearrange(mask, "w b d -> (w b) d")

In [30]:
ab[mask.any(dim=-1)]

tensor([[0.5869, 0.3689, 0.9671, 0.6872, 0.4481, 0.8763, 0.2241, 0.7484],
        [0.9406, 0.8606, 0.8131, 0.7456, 0.8908, 0.8226, 0.6802, 0.3038],
        [0.4385, 0.6148, 0.5822, 0.6618, 0.5690, 0.2471, 0.6140, 0.0323],
        [0.4150, 0.9016, 0.1953, 0.1318, 0.6647, 0.4806, 0.9696, 0.8082],
        [0.8015, 0.4549, 0.6941, 0.1677, 0.7015, 0.1137, 0.7601, 0.2935],
        [0.1857, 0.7594, 0.2515, 0.3138, 0.1438, 0.4808, 0.0077, 0.3284],
        [0.8238, 0.6393, 0.8214, 0.7430, 0.4867, 0.7158, 0.7789, 0.5869],
        [0.4623, 0.7353, 0.0265, 0.2426, 0.7059, 0.7859, 0.0518, 0.1387],
        [0.1538, 0.0783, 0.2859, 0.9093, 0.2125, 0.1035, 0.8887, 0.0912],
        [0.9264, 0.3494, 0.7079, 0.2979, 0.9180, 0.4977, 0.0101, 0.7014]])

In [31]:
ab

tensor([[0.5869, 0.3689, 0.9671, 0.6872, 0.4481, 0.8763, 0.2241, 0.7484],
        [0.9406, 0.8606, 0.8131, 0.7456, 0.8908, 0.8226, 0.6802, 0.3038],
        [0.4385, 0.6148, 0.5822, 0.6618, 0.5690, 0.2471, 0.6140, 0.0323],
        [0.4150, 0.9016, 0.1953, 0.1318, 0.6647, 0.4806, 0.9696, 0.8082],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.8015, 0.4549, 0.6941, 0.1677, 0.7015, 0.1137, 0.7601, 0.2935],
        [0.1857, 0.7594, 0.2515, 0.3138, 0.1438, 0.4808, 0.0077, 0.3284],
        [0.8238, 0.6393, 0.8214, 0.7430, 0.4867, 0.7158, 0.7789, 0.5869],
        [0.4623, 0.7353, 0.0265, 0.2426, 0.7059, 0.7859, 0.0518, 0.1387],
        [0.1538, 0.0783, 0.2859, 0.9093, 0.2125, 0.1035, 0.8887, 0.0912],
        [0.9264, 0.3494, 0.7079, 0.2979, 0.9180, 0.4977, 0.0101, 0.7014]])

In [35]:
torch.zeros([10] + list(a.size())[1:])

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [36]:
list(a.size())[:0]

[]

In [41]:
pad_shape = list(a.size())[:0] + [int(max_rels) - a.size(0)] + list(a.size())[(0 + 1):]

In [42]:
pad_shape

[0, 8]